# Qwen2.5 Tatkal Assistant Fine-tune (Unsloth Edition)

This notebook fine-tunes `Qwen/Qwen2.5-Coder-1.5B-Instruct` using **Unsloth** for faster, memory-efficient training.
It proceeds in two stages: first on the IRCTC Tatkal project code corpus (`project_code.jsonl`), then on the conversational instruction dataset (`chat_dataset.jsonl`).
Finally, it exports the model to **GGUF** format for local use.

In [1]:
# Step 1: Install Unsloth and dependencies
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
# Step 2: Imports and utility setup
import os
import torch
from pathlib import Path
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Step 3: Configuration & Model Loading
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

default_root = Path("/content")
data_root = Path(os.getenv("LLAMA_DATA_ROOT", str(default_root))).expanduser()
textbook_data_file = str((data_root / "project_code.jsonl").expanduser())
workbook_data_file = str((data_root / "chat_dataset.jsonl").expanduser())
new_model_name = "qwen25-coder-tatkal-assistant"
system_prompt = "You are a precise Indian Railways Tatkal booking assistant built on Qwen2.5."

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.11.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
# Step 4: Load and format datasets
textbook_dataset = load_dataset("json", data_files=textbook_data_file, split="train")
textbook_columns = textbook_dataset.column_names

def format_textbook(example):
    def cleaned(value, fallback=""):
        if isinstance(value, str):
            stripped = value.strip()
            return stripped if stripped else fallback
        return fallback

    language = cleaned(example.get("language") or example.get("lang"), "text")
    path = cleaned(example.get("path") or example.get("file") or example.get("module"), f"snippet.{language}")
    body = example.get("content") or example.get("text") or ""
    if isinstance(body, str):
        body = body.strip("\n")
    header = f"// File: {path} ({language})\n" if path else ""
    return {"text": header + body}

textbook_dataset = textbook_dataset.map(format_textbook, remove_columns=textbook_columns)

workbook_dataset = load_dataset("json", data_files=workbook_data_file, split="train")
workbook_columns = workbook_dataset.column_names

def format_workbook(example):
    instruction = (example.get("instruction") or "").strip()
    extra_input = (example.get("input") or "").strip()
    user_block = instruction if not extra_input else f"{instruction}\n\nInput:\n{extra_input}"
    response = (example.get("output") or "").strip()
    return {
        "text": (
            f"<|im_start|>system\n{system_prompt}\n<|im_end|>\n"
            f"<|im_start|>user\n{user_block}\n<|im_end|>\n"
            f"<|im_start|>assistant\n{response}\n<|im_end|>"
        )
    }

workbook_dataset = workbook_dataset.map(format_workbook, remove_columns=workbook_columns)

print(f"Textbook samples: {len(textbook_dataset)}")
print(f"Workbook samples: {len(workbook_dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Textbook samples: 16
Workbook samples: 30


In [5]:
# Step 5: Training Setup

def make_training_args(run_name: str) -> TrainingArguments:
    return TrainingArguments(
        output_dir=f"./results/{run_name}",
        num_train_epochs=10,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=-1,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
    )

def build_trainer(train_dataset, run_name: str, model_obj):
    return SFTTrainer(
        model=model_obj,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=False,
        args=make_training_args(run_name),
    )

print("Starting Stage 1 (Textbook)")
textbook_trainer = build_trainer(textbook_dataset, "stage_textbook", model)
textbook_trainer.train()

# Unsloth models are optimized, so we can continue training directly
print("Starting Stage 2 (Workbook)")
workbook_trainer = build_trainer(workbook_dataset, "stage_workbook", model)
workbook_trainer.train()

Starting Stage 1 (Textbook)


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/16 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16 | Num Epochs = 10 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072 of 1,617,573,376 (4.57% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sidhureddy-legala (sidhureddy-legala-torry-harris-integration-solutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Starting Stage 2 (Workbook)


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/30 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 10 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072 of 1,617,573,376 (4.57% trained)


Step,Training Loss
1,4.564000
2,4.549400
3,4.304300
4,4.124000
5,3.874100
6,3.685100
7,3.306600
8,3.433600
9,2.756300
10,2.626200


TrainOutput(global_step=40, training_loss=2.0369997352361677, metrics={'train_runtime': 65.5689, 'train_samples_per_second': 4.575, 'train_steps_per_second': 0.61, 'total_flos': 264935818752000.0, 'train_loss': 2.0369997352361677, 'epoch': 10.0})

In [6]:
# Step 6: Save and Export to GGUF

# Save LoRA adapters
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
print(f"Saved LoRA adapters to {new_model_name}")

# Export to GGUF
print("Exporting to GGUF...")
model.save_pretrained_gguf(new_model_name, tokenizer, quantization_method = "q4_k_m")
print(f"GGUF model saved to {new_model_name}")

Saved LoRA adapters to qwen25-coder-tatkal-assistant
Exporting to GGUF...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:09<00:00,  9.92s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:10<00:00, 10.67s/it]


Unsloth: Merge process complete. Saved to `/content/qwen25-coder-tatkal-assistant`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['qwen2.5-coder-1.5b

'/content'

### Step 1: Ensure `huggingface_hub` is Installed

Make sure the necessary library for interacting with Hugging Face Hub is installed.

In [9]:
%%capture
!pip install huggingface_hub

### Step 2: Log in to Hugging Face

Run the following command to log in to Hugging Face. You'll be prompted to enter your Hugging Face token. Ensure the token has **write access** to create and upload to repositories.

In [13]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

### Step 3: Provide Your Hugging Face Username

Enter your Hugging Face username below. This will be used to construct the name of your model repository.

In [14]:
from huggingface_hub import HfApi, create_repo
import os

hf_username = input("Please enter your Hugging Face username: ")
if not hf_username:
    raise ValueError("Hugging Face username cannot be empty.")

print(f"Hugging Face username set to: {hf_username}")

# Existing variables from the notebook
# new_model_name = "qwen25-coder-tatkal-assistant"
# This repository name indicates it's an Ollama-compatible model
repo_id = f"{hf_username}/{new_model_name}-ollama"

gguf_file_name = "qwen2.5-coder-1.5b-instruct.Q4_K_M.gguf"
# Correcting the path: GGUF file is likely in the root /content/ directory
gguf_file_path = f"./{gguf_file_name}"

modelfile_name = "Modelfile"
modelfile_path = f"./{modelfile_name}"

api = HfApi()

print(f"Target Hugging Face repository: {repo_id}")
print(f"GGUF file to upload: {gguf_file_path}")
print(f"Modelfile to upload: {modelfile_path}")

Please enter your Hugging Face username: kanyaraasi
Hugging Face username set to: kanyaraasi
Target Hugging Face repository: kanyaraasi/qwen25-coder-tatkal-assistant-ollama
GGUF file to upload: ./qwen2.5-coder-1.5b-instruct.Q4_K_M.gguf
Modelfile to upload: ./Modelfile


### Step 4: Create Repository and Upload Files

This cell will create a new repository on Hugging Face (if it doesn't already exist) and then upload both your `.gguf` model file and the `Modelfile`.

In [15]:
try:
    create_repo(repo_id=repo_id, repo_type="model", private=False)
    print(f"Repository '{repo_id}' created successfully.")
except Exception as e:
    if "already exists" in str(e):
        print(f"Repository '{repo_id}' already exists. Proceeding with upload.")
    else:
        print(f"Could not create repository '{repo_id}': {e}")

# Upload GGUF file
if os.path.exists(gguf_file_path):
    print(f"Uploading GGUF file '{gguf_file_name}' to {repo_id}...")
    api.upload_file(
        path_or_fileobj=gguf_file_path,
        path_in_repo=gguf_file_name,
        repo_id=repo_id,
        repo_type="model"
    )
    print(f"Successfully uploaded {gguf_file_name}.")
else:
    print(f"Error: GGUF file not found at {gguf_file_path}")

# Upload Modelfile
if os.path.exists(modelfile_path):
    print(f"Uploading Modelfile '{modelfile_name}' to {repo_id}...")
    api.upload_file(
        path_or_fileobj=modelfile_path,
        path_in_repo=modelfile_name,
        repo_id=repo_id,
        repo_type="model"
    )
    print(f"Successfully uploaded {modelfile_name}.")
else:
    print(f"Error: Modelfile not found at {modelfile_path}")

print(f"\nAll specified files have been uploaded to: https://huggingface.co/{repo_id}/tree/main")
print(f"Users can now clone this repository and use it with Ollama by navigating to the cloned directory and running: `ollama create {new_model_name}-ollama -f ./Modelfile`")

Could not create repository 'kanyaraasi/qwen25-coder-tatkal-assistant-ollama': 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-691f1545-137bfc63542c0f15280b8811;c80c9905-bf45-46f6-8616-960d28768ccf)

You already created this model repo: kanyaraasi/qwen25-coder-tatkal-assistant-ollama
Uploading GGUF file 'qwen2.5-coder-1.5b-instruct.Q4_K_M.gguf' to kanyaraasi/qwen25-coder-tatkal-assistant-ollama...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...1.5b-instruct.Q4_K_M.gguf:   0%|          |  548kB /  986MB            

Successfully uploaded qwen2.5-coder-1.5b-instruct.Q4_K_M.gguf.
Uploading Modelfile 'Modelfile' to kanyaraasi/qwen25-coder-tatkal-assistant-ollama...


No files have been modified since last commit. Skipping to prevent empty commit.


Successfully uploaded Modelfile.

All specified files have been uploaded to: https://huggingface.co/kanyaraasi/qwen25-coder-tatkal-assistant-ollama/tree/main
Users can now clone this repository and use it with Ollama by navigating to the cloned directory and running: `ollama create qwen25-coder-tatkal-assistant-ollama -f ./Modelfile`
